In [1]:
import matplotlib.pyplot as plt
import numpy as np
import egg.core as core
import pandas as pd

from utils.analysis_tools_lazimpa import *
from utils.analysis_from_interaction import *
from archs import Sender, Receiver
from train import loss
from models_lazimpa import LazImpaSenderReceiverRnnGS
from dataset import DataSet
import torch.nn.functional as F

In [ ]:
datasets = ('(3,4)', '(3,8)', '(4,4)')
n_attributes = (3, 3, 4)
n_values = (4, 8, 4)
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]
run = [0,1,2,3,4]

In [2]:
attributes = 3
values = 4
n_epochs = 300
path = 'results/(3,4)_game_size_10_vsf_3/'
run = 0

In [3]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = True
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting

'lazimpa_context_aware'

In [4]:
# Start what kind of variables do I need for attribute visualization

interaction = load_interaction(path,setting,run,n_epochs)
concepts, context_condition = retrieve_concepts_context(interaction,values)
messages = retrieve_messages(interaction,False,remove_after_eos=True)
message_length = MessageLengthHierarchical.compute_message_length(messages)
unique_messages, frequencies = torch.unique(messages,dim=0,return_counts=True) # frequency of message outside of this usage case
unique_length = MessageLengthHierarchical.compute_message_length(unique_messages)
percent_wrong, bool_wrong = percent_wrong_from_interaction(interaction)
listener,_,model = load_listener(path, setting, run, attributes, values, context_unaware)
sender = model.sender 

m = [f"{ *m, }" for m in messages.numpy()] # as str for hashing
l = message_length.numpy()
cx = context_condition
fint = [list(map(int,list(f))) for c,f in concepts]
fix = [f"{v}" for v in fint] # fix as string
c = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concepts]
um = [f"{ *m, }" for m in unique_messages.numpy()]
ul = unique_length.numpy()
f = frequencies.numpy()

# make one DataFrame containing all the data we have
in_list = []
for i in range(message_length.shape[0]):
    content = [int(sl) if sl == 1. else "" for sl in list(fint[i])]
    content.extend((fix[i],fint[i], np.where(concepts[i][1]==1.0)[0].tolist(), sum(fint[i]), c[i], cx[i], m[i], messages.numpy()[i], l[i],percent_wrong[i].item(),bool_wrong[i].item()))
    in_list.append(content)
df = pd.DataFrame(in_list,columns=["fixed 1", "fixed 2", "fixed 3","fixed", "fixed int","which fixed indices", "nr attributes", "concept", "context condition", "message", "message int", "message length", "wrong %", "wrong some"])
unique_df = pd.DataFrame([(m,f) for m,f in zip(list(um),list(f))],columns=["message", "usage frequency"])
df = df.merge(unique_df,how='right',on="message")
df

results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1762, 20, 12])


,fixed 1,fixed 2,fixed 3,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message int,message length,wrong %,wrong some,usage frequency
0,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 2, 3]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,25,1,229
1,,1,1,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 0, 3]",0,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229
2,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 3, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,5,1,229
3,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 2, 3]",0,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229
4,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[1, 2, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10
1758,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10
1759,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10
1760,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10


In [5]:
# remove all values with wrong some != 0 # in test removes ca 350 from 1784 rows, in real only 183 left !!!
df_correct = df[df['wrong some'] == 0]

# adjust frequencies
unique_messages, frequencies = torch.unique(torch.Tensor(df_correct['message int'].array),dim=0,return_counts=True) # frequency of message outside of this usage case
f_c = frequencies.numpy()
um_c = [f"{ *m, }" for m in unique_messages.numpy().astype(np.int32)]
ul_c = unique_length.numpy()
unique_df = pd.DataFrame([(m,f_c) for m,f_c in zip(list(um_c),list(f))],columns=["message", "correct frequency"])
df_adjusted = df_correct.merge(unique_df,how='right',on="message")
df_adjusted

C:\Users\eosan\AppData\Local\Temp\ipykernel_10616\3096840174.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  unique_messages, frequencies = torch.unique(torch.Tensor(df_correct['message int'].array),dim=0,return_counts=True) # frequency of message outside of this usage case


,fixed 1,fixed 2,fixed 3,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message int,message length,wrong %,wrong some,usage frequency,correct frequency
0,,1,1,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 0, 3]",0,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229,229
1,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 2, 3]",0,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229,229
2,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[1, 2, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229,229
3,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229,229
4,1,,1,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[3, -1, 3]",0,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,229,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10,6
1367,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10,6
1368,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10,6
1369,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 0]",2,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,10,6


# Meaning calculation attempt

In [6]:
def many_hot(input,data):
    output = torch.zeros([sum(data.properties_dim)])
    start = 0

    for elem, dim in zip(input, data.properties_dim):
        output[start + elem] = 1
        start += dim

    return output

In [7]:
def get_sender_input(data, concept_idx, context_condition, encoding_func):
        sender_concept, sender_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        sender_targets = sender_concept[0]
        sender_input = [obj for obj in sender_targets]

        # append context objects
        # get context of relevant context condition
        for distractor_objects, context_cond in sender_context:
            if context_cond == context_condition:
                # add distractor objects for the sender
                for obj in distractor_objects:
                    sender_input.append(obj)

        # ENCODE and return as TENSOR
        sender_input = torch.stack([many_hot(elem,data) for elem in sender_input])
        return sender_input

In [8]:
def get_receiver_input(data, concept_idx, context_condition, encoding_func):
        receiver_concept, receiver_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        receiver_targets = receiver_concept[0]
        receiver_input = [obj for obj in receiver_targets]

        for distractor_objects, context_cond in receiver_context:
            if context_cond == context_condition:
                # add distractor objects for the receiver
                for obj in distractor_objects:
                    receiver_input.append(obj)
        
        random.shuffle(receiver_input)
        receiver_label = [idx for idx, obj in enumerate(receiver_input) if obj in receiver_targets]
        receiver_label = torch.Tensor(receiver_label).to(torch.int64)
        receiver_label = F.one_hot(receiver_label, num_classes=data.game_size * 2).sum(dim=0).float()

        # ENCODE and return as TENSOR
        receiver_input = torch.stack([many_hot(elem,data) for elem in receiver_input])
        return receiver_label, receiver_input

In [9]:
# calculate meanings (concept x message for fine context)
runs = 7
data = DataSet(list(itertools.repeat(values, attributes)),
    game_size=10,
    scaling_factor=10,
    zero_shot=True,
    zero_shot_test=False,
    sample_context=False)

concept_fixed_set = data.get_all_concepts()
set_concepts = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concept_fixed_set]
fint_set = [list(map(int,list(f))) for c,f in concept_fixed_set]
sender_input = torch.cat([get_sender_input(data, i, sum(f)-1, data._many_hot_encoding).unsqueeze(0) for i,f in zip(range(len(concept_fixed_set)),fint_set)],dim=0)
messages_original = [sender(sender_input) for r in range(runs)]
messages_tensor = torch.cat([messages_without_after_eos(msg,0.0).argmax(dim=-1).unsqueeze(1) for msg in messages_original],dim=1)

Creating train_ds, val_ds and test_ds...


100%|██████████| 124/124 [00:00<00:00, 123979.43it/s]

Length of train and validation datasets: 0 / 0
Length of test dataset: 0


In [10]:
# get whether the listener can understand the messages
receiver_input_list = []
labels_list = []
for i,f in zip(range(len(concept_fixed_set)),fint_set):
    labels, receiver_input = get_receiver_input(data, i, sum(f)-1, data._many_hot_encoding)
    receiver_input_list.append(receiver_input.unsqueeze(0))
    labels_list.append(labels.unsqueeze(0).unsqueeze(0).unsqueeze(0))

receiver_input = torch.cat(receiver_input_list,dim=0)
labels = torch.cat(labels_list,dim=0).repeat(1,runs,attributes + 1, 1)

eos_mask_total = 0.0 != messages_tensor # remember this is fore several runs
# only values before eos, to switch
eos_excluded_cumulative = torch.cat([ eos_mask_total[:,:,:i+1].sum(dim=2).unsqueeze(-1) == i+1 for i in range(attributes +1) ],dim=-1)
# until eos, evaluating changes
eos_included_cumulative = torch.cat([torch.ones_like(eos_excluded_cumulative[:,:,-1]).unsqueeze(-1).bool(), eos_excluded_cumulative[:,:,:-1]],dim=-1)
eos_exact = (eos_included_cumulative != eos_mask_total) # only symbol first eos

predictions = torch.cat([(listener(msg,receiver_input) > 0).float().unsqueeze(1) for msg in messages_original],dim=1)
# remove after eos prediction
predictions = torch.where(eos_exact.unsqueeze(-1),predictions,labels)
score = (predictions != labels)

In [11]:
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = sum(list(message_counts.values()))
    return unique_messages, frequencies

In [11]:
# make one DataFrame containing all the concepts and messages used for them in fine context

# make data hashable, and calc averages etc. 
messages_set = [list(set([f"{ *mes, }" for mes in msg_5])) for msg_5 in [msg.tolist() for msg in messages_tensor]]
messages_str = [[f"{ *mes, }" for mes in msg_5] for msg_5 in [msg.tolist() for msg in messages_tensor]]
concepts_str = [f"{ *con, }" for con in set_concepts]
wrong_in_total = score.sum(dim=(2,3)).bool().sum(dim=-1)/runs
wrong_per_message = score.sum(dim=(2,3))/20

# create DataFrame to calc average wrongness per message etc
in_list = []
for i in range(len(messages_str)):
    for j,m in enumerate(messages_str[i]):
        in_list.append((m,concepts_str[i],wrong_per_message[i][j].item()))
meaning_message_df = pd.DataFrame(in_list,columns=["message","meaning","wrong %"])
meaning_message_df = meaning_message_df.groupby(['meaning','message'])['wrong %'].apply(lambda l: l.mean()).reset_index().round(2)
with_messages = meaning_message_df.groupby(['meaning'])['message'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_wrong = meaning_message_df.groupby(['meaning'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
wrong_per_message_df = with_messages.merge(with_wrong,how='right',on="meaning")

# create DataFrame with all data
in_list = []
for i in range(len(messages_set)):
    in_list.append((concepts_str[i],fint_set[i],wrong_in_total[i].item()))
meaning_df = pd.DataFrame(in_list,columns=["meaning","fixed meaning int","not all correct percent total"]).round(1)
#meaning_df.round(1).drop(meaning_df[meaning_df["not all correct percent total"] == 0.0].index)
meaning_df = meaning_df.merge(wrong_per_message_df,how='right',on='meaning')
meaning_df

,meaning,fixed meaning int,not all correct percent total,message,wrong %
0,"(-1, -1, 0)","[0, 0, 1]",0.6,"[(10, 0, 0, 0), (10, 1, 0, 0), (15, 0, 0, 0)]","[0.13, 0.01, 0.3]"
1,"(-1, -1, 1)","[0, 0, 1]",0.1,"[(2, 0, 0, 0)]",[0.01]
2,"(-1, -1, 2)","[0, 0, 1]",0.0,"[(14, 0, 0, 0)]",[0.0]
3,"(-1, -1, 3)","[0, 0, 1]",1.0,"[(0, 0, 0, 0)]",[0.3]
4,"(-1, 0, -1)","[0, 1, 0]",0.0,"[(7, 7, 0, 0), (7, 7, 7, 0)]",[0.0]
...,...,...,...,...,...
119,"(3, 3, -1)","[1, 1, 0]",0.9,"[(12, 7, 0, 0), (7, 0, 0, 0), (7, 12, 0, 0)]","[0.05, 0.04]"
120,"(3, 3, 0)","[1, 1, 1]",0.0,"[(9, 12, 0, 0)]",[0.0]
121,"(3, 3, 1)","[1, 1, 1]",0.4,"[(12, 0, 0, 0), (12, 9, 0, 0), (7, 12, 0, 0)]","[0.0, 0.3]"
122,"(3, 3, 2)","[1, 1, 1]",1.0,"[(12, 0, 0, 0), (12, 7, 0, 0), (14, 0, 0, 0), ...",[0.65]


In [12]:
# create a similar dataframe but sorted by message
with_wrong = meaning_message_df.groupby(['message'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_meaning = meaning_message_df.groupby(['message'])['meaning'].apply(aggregate_messages).reset_index()
with_meaning[['unique meanings', 'frequency used by meaning']] = pd.DataFrame(with_meaning['meaning'].tolist(), index=with_meaning.index)
message_df = with_meaning.merge(with_wrong,how='right',on="message").drop("meaning",axis=1)
message_df

,message,unique meanings,frequency used by meaning,wrong %
0,"(0, 0, 0, 0)","[(-1, -1, 3), (1, 2, 3), (3, 2, 3), (3, 1, 3),...",25,"[0.1, 0.05, 0.14, 0.0, 0.17, 0.3, 0.11, 0.16, ..."
1,"(1, 0, 0, 0)","[(-1, 0, 0), (-1, 1, 0), (-1, 2, 0), (1, -1, -...",13,"[0.1, 0.05, 0.38, 0.3, 0.28, 0.64, 0.06, 0.13,..."
2,"(1, 1, 0, 0)","[(-1, 1, 0)]",1,[0.2]
3,"(1, 12, 0, 0)","[(3, -1, 0)]",1,[0.0]
4,"(1, 2, 0, 0)","[(-1, 1, 0), (1, 1, 0)]",2,"[0.2, 0.12]"
...,...,...,...,...
65,"(9, 11, 0, 0)","[(0, 3, 0)]",1,[0.0]
66,"(9, 12, 0, 0)","[(1, 3, 1), (3, 3, 0)]",2,[0.0]
67,"(9, 15, 0, 0)","[(2, 3, 0)]",1,[0.0]
68,"(9, 2, 0, 0)","[(0, 3, 1), (1, 3, 1), (2, 3, 1)]",3,"[0.03, 0.1]"


In [ ]:
# find true meaning


# Data Analysis

In [13]:
message_length.float().mean() # weighted by occurence

tensor(1.1198)

In [14]:
# How long are the messages on average used for a certain fix attribute number # TODO frequency??
scores = message_length_per_hierarchy_level(interaction, attributes)
scores # These values are correct!

message length tensor([1, 1, 1,  ..., 1, 2, 1])
hierarchical [1.402, 1.076, 1.122]


[1.402, 1.076, 1.122]

In [15]:
# How long are the messages on average used in the different context conditions # TODO add frequency
grouped_by_context = df.groupby(["context condition"])['message length'].apply(lambda l: l.mean()).reset_index()
grouped_by_context_wrong = df.groupby(["context condition"])['wrong %'].apply(lambda l: l.mean()).reset_index()
grouped_by_context_wrong2 = df.groupby(["context condition"])['wrong some'].apply(lambda l: l.mean()).reset_index()
grouped_by_context = grouped_by_context.merge(grouped_by_context_wrong,how='right',on='context condition')
grouped_by_context = grouped_by_context.merge(grouped_by_context_wrong2,how='right',on='context condition')
grouped_by_context

,context condition,message length,wrong %,wrong some
0,0,0.948468,0.988858,0.054318
1,1,1.120301,3.593985,0.282707
2,2,1.443272,5.554090,0.432718


In [16]:
# how long are the messages on average for different numbers of fixed attributes # TODO add frequency
grouped_by_nr_fixed_attributes = df.groupby(["nr attributes","context condition"])["message length"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes_wrong = df.groupby(["nr attributes","context condition"])["wrong %"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes_wrong2 = df.groupby(["nr attributes","context condition"])["wrong some"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes = grouped_by_nr_fixed_attributes.merge(grouped_by_nr_fixed_attributes_wrong,how='right',on=['nr attributes', 'context condition'])
grouped_by_nr_fixed_attributes = grouped_by_nr_fixed_attributes.merge(grouped_by_nr_fixed_attributes_wrong2,how='right',on=['nr attributes', 'context condition'])
grouped_by_nr_fixed_attributes

,nr attributes,context condition,message length,wrong %,wrong some
0,1,0,1.402439,5.975610,0.353659
1,2,0,0.931373,0.539216,0.026144
2,2,1,1.226351,5.996622,0.388514
3,3,0,0.851515,0.166667,0.006061
4,3,1,1.035230,1.666667,0.197832
5,3,2,1.443272,5.554090,0.432718


In [17]:
# how often are messages used with different lengths in total
#grouped_by_length = pd.DataFrame([(l,f) for l,f in zip(ul,f)],columns=["length","frequency"])
grouped_by_length = df.groupby(["message length"])['usage frequency'].apply(lambda f: (f.mean(),f.std(ddof=False))).reset_index()
grouped_by_length_wrong = df.groupby(["message length"])['wrong %'].apply(lambda f: f.mean()).reset_index()
grouped_by_length_wrong2 = df.groupby(["message length"])['wrong some'].apply(lambda f: f.mean()).reset_index()
grouped_by_length[['mean usage frequency', 'std usage frequency']] = pd.DataFrame(grouped_by_length['usage frequency'].tolist(), index=grouped_by_length.index)
grouped_by_length = grouped_by_length.merge(grouped_by_length_wrong,how='right',on='message length')
grouped_by_length = grouped_by_length.merge(grouped_by_length_wrong2,how='right',on='message length')
grouped_by_length = grouped_by_length.drop('usage frequency',axis=1)
grouped_by_length

,message length,mean usage frequency,std usage frequency,wrong %,wrong some
0,0,229.000000,0.000000,8.515284,0.537118
1,1,85.261343,27.483219,1.796733,0.160617
2,2,8.094787,4.870894,2.890995,0.210900
3,3,1.222222,0.415740,6.111111,0.222222


In [12]:
df_attributes = df[df['context condition'] + 1 == df['nr attributes']]
# Idee: remove all messages with wrong some, so that wrongly used messages or estimations are excluded. Because they mean something different. 
#df_attributes = df_attributes[df_attributes['wrong some'] == 0] # Done at the top, if not there you can use it here. 
df_attributes = df

In [13]:
df_attributes_grouped = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['message length'].apply(lambda l: l.mean()).reset_index()
df_attributes_frequency_total = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['usage frequency'].apply(lambda l: l.sum()).reset_index().rename(columns={'usage frequency':'usage frequency total'})
df_attributes_frequency_mean = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['usage frequency'].apply(lambda l: l.mean()).reset_index().rename(columns={'usage frequency':'usage frequency mean'})
df_attributes_wrong = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['wrong %'].apply(lambda l: l.mean()).reset_index()
df_attributes_wrong2 = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['wrong some'].apply(lambda l: l.mean()).reset_index()
df_attributes = df_attributes_grouped.merge(df_attributes_frequency_total,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
#df_attributes = df_attributes_grouped.merge(df_attributes_frequency_mean,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes = df_attributes.merge(df_attributes_wrong,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes = df_attributes.merge(df_attributes_wrong2,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes

,fixed 1,fixed 2,fixed 3,message length,usage frequency total,wrong %,wrong some
0,1,1,1,1.122449,90690,2.574212,0.221707
1,1,1,,1.216346,12943,2.451923,0.221154
2,1,,1,0.946903,25282,5.044248,0.238938
3,1,,,1.562500,1526,9.218750,0.500000
4,,1,1,1.077381,15584,1.726190,0.136905
5,,1,,1.761905,319,2.380952,0.142857
6,,,1,0.965517,3482,5.000000,0.344828
